In [3]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.models import load_model


tf.device('/gpu:1')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.compat.v1.Session(config=config)

import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import PreProcessing
import net
import PatchExtractor

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Physical devices cannot be modified after being initialized
GPU selected: 0
Physical devices cannot be modified after being initialized


In [2]:
in_path = 'giuriati_2/20170621_deg0_HHVV.npy'
out_path = 'cnn_article'
architecture = 'Auto3D2'
ny = 3 
data_augmentation = True
preprocessing = 'normalize'
patch_size = 64  ##64
patch_stride = 4 ##4
n_bsc = 5 #5 

In [3]:
def parse_pp(string):
    return getattr(PreProcessing, string)

def parse_net(string):
    return getattr(net, string)

In [4]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

field, campaign = in_path.split('/')
campaign, extension = campaign.split('.')

In [5]:
dataset = np.load('./datasets/'+str(in_path),allow_pickle=True).item()

train_bsc_idx = np.where(np.asarray(dataset['ground_truth']) == 0)[0][:n_bsc]
trainset = dataset['data'][train_bsc_idx]
trainset = np.moveaxis(trainset, np.argmin(trainset.shape), -1)
del dataset


if patch_size is not None:
    patch_size = (patch_size, patch_size)
else:
    patch_size = trainset.shape[1:]

patch_size = patch_size + (ny,)
patch_stride = (patch_stride, patch_stride, 1)

In [6]:
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm
training = './cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3'
net_weights = training + '.h5'


In [7]:
train_path = in_path
dataset = np.load('./datasets/' + str(in_path),allow_pickle=True).item()

data = dataset['data']

pe = PatchExtractor.PatchExtractor(patch_size, stride=patch_stride)

gt = np.asarray(dataset['ground_truth'])
del dataset

test_idx = np.arange(data.shape[0])

if in_path == train_path:
    train_idx = np.where(gt == 0)[0][:n_bsc]
    test_idx = np.delete(test_idx, train_idx)
testset = data[test_idx]
gt = gt[test_idx]
del data
xtrain = np.moveaxis(testset, np.argmin(testset.shape), -1)
ytrain=gt


In [8]:
xtrain=xtrain.reshape(61,170,440,1)
xtrain.shape

(61, 170, 440, 1)

In [13]:
ytrain=ytrain.astype('float32')

In [23]:
ytrain=ytrain.reshape(61)

In [4]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(170,440,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 170, 440, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 168, 438, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 84, 219, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 84, 219, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 219, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 82, 217, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 108, 64)       0

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(xtrain, ytrain, batch_size=5, epochs=15, verbose=1)


In [31]:
model.evaluate(xtrain, ytrain)

2/2 [==============================] - 18s 9s/step - loss: 0.2112 - accuracy: 0.8852


[0.21119092404842377, 0.8852459192276001]

In [32]:
model.save("model.h5")

In [35]:
model = load_model('model.h5')

In [36]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 170, 440, 32)      320       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 168, 438, 32)      9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 84, 219, 32)       0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 84, 219, 32)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 84, 219, 64)       18496     
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 82, 217, 64)       36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 41, 108, 64)      

In [37]:
model.predict(xtrain)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [38]:
ytrain

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)